In [1]:
import sys
from pathlib import Path

PIPELINE_ROOT = Path('.').absolute().parent
sys.path.append(PIPELINE_ROOT.as_posix())

In [2]:
import numpy as np
import requests

from utilities.sqlcontroller import SqlController

Connecting dklab@db.dk.ucsd.edu:3306


In [5]:
def get_transform(animal):
    res = requests.get(f'https://activebrainatlas.ucsd.edu/activebrainatlas/alignatlas?animal={animal}')
    r = np.array(res.json()['rotation'])
    t = np.array(res.json()['translation'])
    return r, t

def transform_point_dataframe(
    src_url_id=182,
    src_animal='DK52',
    src_scale=(0.325, 0.325, 20),
    dst_animal='DK39',
    dst_scale=(0.325, 0.325, 20),
    layer='premotor'
):
    """Transform coordinates from one animal brain to another.
    
    The transformation is from src coordinates to dst coordinates. 
    """
    src_scale = np.array(src_scale)
    dst_scale = np.array(dst_scale)

    # Get transformations from atlas to each animal
    r_src, t_src = get_transform(src_animal)
    r_dst, t_dst = get_transform(dst_animal)

    # Get transformation from src to dst
    r = r_dst @ np.linalg.inv(r_src)
    t_phys = np.diag(dst_scale) @ t_dst - np.diag(src_scale) @ t_src

    # Get src points data
    sqlController = SqlController(src_animal)
    df_src = sqlController.get_point_dataframe(src_url_id)
    df_src = df_src[df_src['Layer'] == layer]

    # Transform points from src to dst
    x_src = df_src[['X', 'Y', 'Section']].to_numpy().T
    x_src_phys = np.diag(src_scale) @ x_src
    x_dst_phys = r @ x_src_phys + t_phys
    x_dst = np.diag(1 / dst_scale) @ x_dst_phys
    return x_dst

In [6]:
x_dst = transform_point_dataframe()
x_dst.T

array([[29813.51783482, 19724.424011  ,   235.45752087],
       [30305.897857  , 18557.92565718,   227.92850872],
       [30900.35589941, 18902.60971262,   225.95614745],
       ...,
       [30363.71514167, 19000.82435774,   228.060085  ],
       [38438.55267167, 19657.49554538,   272.55131747],
       [38819.48106832, 20039.77589175,   273.20326571]])